In [1]:
import sys 
sys.path.append('/home/monte.flora/python_packages/wofs_ml_severe')
from wofs_ml_severe.data.ml_2to6_data_pipeline import (GridPointExtracter,
                                                       subsampler, 
                                                       load_dataset)
from os.path import join

In [2]:
dates = ['20180530']

In [3]:
##date = '20210504'

init_time = '1900'

for date in dates:
    path = f'/work/mflora/SummaryFiles/{date}/{init_time}'

    out_path = join('/work/mflora/ML_2TO6HR', 'data') 

    X_env, X_strm, ncfile  = load_dataset(path)
    extracter = GridPointExtracter(ncfile, env_vars=X_env.keys(), strm_vars=X_strm.keys())
    df = extracter(X_env, X_strm, predict=True)
    df.reset_index(drop=True, inplace=True)

    out_name = join(out_path, f'wofs_ml_severe__2to6hr__{date}_data.feather')
    print(f'Saving {out_name}...')
    df.to_feather(out_name)

Saving /work/mflora/ML_2TO6HR/data/wofs_ml_severe__2to6hr__20180530_data.feather...
